In [1]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\aditya\Documents


In [2]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

def rawProcess(inputFile='C:/Users/aditya/Documents/gData/train.csv', rowCount=None):
    splitCol = ['device', 'geoNetwork', 'totals', 'trafficSource']

    table = pd.read_csv(inputFile, converters={cName: json.loads for cName in splitCol}, dtype={'fullVisitorId': 'str'}, nrows=rowCount)

    for cName in splitCol:
        tmpCol = json_normalize(table[cName])
        tmpCol.columns = [f"{cName}.{innerCol}" for innerCol in tmpCol.columns]
        table = table.drop(cName, axis=1).merge(tmpCol, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(inputFile)}. Shape: {table.shape}")
    return table


print(os.listdir("C:/Users/aditya/Documents/gData"))

['test.csv', 'test_v2.csv', 'train.csv', 'train_v2.csv']


In [3]:
trainSet = rawProcess(rowCount=200000)

Loaded train.csv. Shape: (200000, 55)


In [4]:
dropCol = [col for col in trainSet.columns if trainSet[col].nunique(dropna=False) == 1]

In [5]:
dropCol

['socialEngagementType',
 'device.browserSize',
 'device.browserVersion',
 'device.flashVersion',
 'device.language',
 'device.mobileDeviceBranding',
 'device.mobileDeviceInfo',
 'device.mobileDeviceMarketingName',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.operatingSystemVersion',
 'device.screenColors',
 'device.screenResolution',
 'geoNetwork.cityId',
 'geoNetwork.latitude',
 'geoNetwork.longitude',
 'geoNetwork.networkLocation',
 'totals.visits',
 'trafficSource.adwordsClickInfo.criteriaParameters']

In [6]:
testSet=rawProcess('C:/Users/aditya/Documents/gData/test.csv', rowCount=200000)

Loaded test.csv. Shape: (200000, 53)


In [7]:
print( set(trainSet.columns).difference(set(testSet.columns)))

{'totals.transactionRevenue', 'trafficSource.campaignCode'}


In [8]:
trainSet = trainSet.drop(dropCol+['trafficSource.campaignCode']+['sessionId'], axis=1)

In [9]:
trainSet.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1472830385,1,1472830385,Chrome,desktop,False,Windows,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,1472880147,1,1472880147,Firefox,desktop,False,Macintosh,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,1472865386,1,1472865386,Chrome,desktop,False,Windows,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,1472881213,1,1472881213,UC Browser,desktop,False,Linux,...,NaN,NaN,NaN,NaN,(not set),NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,1472822600,2,1472822600,Chrome,mobile,True,Android,...,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google


In [10]:
testSet = testSet.drop(dropCol + ['sessionId'], axis=1)

In [11]:
trainSet['totals.transactionRevenue'].fillna(0, inplace=True)

In [12]:
from sklearn import preprocessing
encodeCols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']
for column in encodeCols:
    print(column)
    labelEnc = preprocessing.LabelEncoder()
    labelEnc.fit(list(trainSet[column].values.astype('str')) + list(testSet[column].values.astype('str')))
    trainSet[column] = labelEnc.transform(list(trainSet[column].values.astype('str')))
    testSet[column] = labelEnc.transform(list(testSet[column].values.astype('str')))


numCols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces', 'totals.newVisits']    
for column in numCols:
    trainSet[column] = trainSet[column].astype(float)
    testSet[column] = testSet[column].astype(float)

channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect


In [13]:
testSet=testSet.dropna()
trainSet=trainSet.dropna()
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(trainSet,trainSet['totals.transactionRevenue'] , test_size=0.2)

#x_train = trainSet.drop(['totals.transactionRevenue'], axis=1)
#y_train = trainSet['totals.transactionRevenue']
#y_train=y_train.astype('int')
#x_test=testSet
#y_test = testSet['totals.transactionRevenue']
#y_test=y_test.astype('int')


In [19]:
mlp = MLPRegressor(hidden_layer_sizes=(100,100,100), max_iter=1000, alpha=0.0001,
                     solver='adam')

In [20]:
mlp.fit(x_train, y_train)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [21]:
y_pred = mlp.predict(x_test)

In [22]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score

print("mse: ",mean_squared_error(y_test, y_pred),
"explained variance: ",explained_variance_score(y_test, y_pred),
"r2 score: ",r2_score(y_test, y_pred))

mse:  2.8425410078837114e+30 explained variance:  0.0 r2 score:  0.0


In [30]:
from sklearn.ensemble import RandomForestRegressor
rfc = RandomForestRegressor(max_features=2, min_samples_split=4, n_estimators=50, min_samples_leaf=2)

C:\Users\aditya\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [31]:
rfc.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=2, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=2,
           min_samples_split=4, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [32]:
y_pred2 = rfc.predict(x_test)

In [33]:
y_pred2

array([0., 0., 0., ..., 0., 0., 0.])

In [34]:
print("mse: ",mean_squared_error(y_test, y_pred2),
"explained variance: ",explained_variance_score(y_test, y_pred2),
"r2 score: ",r2_score(y_test, y_pred2))

mse:  0.0 explained variance:  1.0 r2 score:  1.0


In [35]:
from sklearn.ensemble import GradientBoostingRegressor

gbc=GradientBoostingRegressor(loss='ls', learning_rate=0.0001, n_estimators=50, max_features='log2', min_samples_split=2, max_depth=1)

In [36]:
gbc.fit(x_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.0001, loss='ls', max_depth=1,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=50, presort='auto',
             random_state=None, subsample=1.0, verbose=0, warm_start=False)

In [37]:
y_pred3=gbc.predict(x_test)

In [38]:
y_pred3

array([0., 0., 0., ..., 0., 0., 0.])

In [39]:
print("mse: ",mean_squared_error(y_test, y_pred3),
"explained variance: ",explained_variance_score(y_test, y_pred3),
"r2 score: ",r2_score(y_test, y_pred3))

mse:  0.0 explained variance:  1.0 r2 score:  1.0


In [24]:
from sklearn.svm import SVR
svr=SVR(kernel= 'poly', C= 1e3, degree= 2)

In [25]:
svr.fit(x_train,y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='auto',
  kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [28]:
y_pred4=svr.predict(x_test)

In [29]:
print("mse: ",mean_squared_error(y_test, y_pred4),
"explained variance: ",explained_variance_score(y_test, y_pred4),
"r2 score: ",r2_score(y_test, y_pred4))

mse:  0.0 explained variance:  1.0 r2 score:  1.0


In [40]:
xTrain = trainSet.drop(['totals.transactionRevenue'], axis=1)
yTrain = trainSet['totals.transactionRevenue']
yTrain=yTrain.astype('int')
xTest=testSet
#yTest = testSet['totals.transactionRevenue']
#yTest=yTest.astype('int')

In [41]:
mlp.fit(xTrain, yTrain)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [42]:
mlpPred=mlp.predict(xTest)

In [43]:
rfc.fit(xTrain,yTrain)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=2, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=2,
           min_samples_split=4, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [44]:
rfcPred=mlp.predict(xTest)

In [45]:
gbc.fit(xTrain, yTrain)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.0001, loss='ls', max_depth=1,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=50, presort='auto',
             random_state=None, subsample=1.0, verbose=0, warm_start=False)

In [46]:
gbcPred=gbc.predict(xTest)

In [47]:
svr.fit(xTrain, yTrain)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='auto',
  kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [48]:
svrPred=svr.predict(xTest)

In [68]:
    def testOut(input):
        tempFrame = pd.DataFrame({"fullVisitorId": testSet["fullVisitorId"].values})
        tempFrame["PredictedLogRevenue"] = np.abs(np.expm1(input))
        tempFrame = tempFrame.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
        tempFrame.columns = ["fullVisitorId", "PredictedLogRevenue"]
        print(tempFrame)

In [69]:
testOut(gbcPred)

             fullVisitorId  PredictedLogRevenue
0      0000059488412965267                  0.0
1      0000118334805178127                  0.0
2      0000174453501096099                  0.0
3      0000397214032106948                  0.0
4      0000421850492821864                  0.0
5      0000955020478109523                  0.0
6      0000998759681663618                  0.0
7      0001059349366430257                  0.0
8      0001251773762451924                  0.0
9      0001269799921883506                  0.0
10      000140003762682805                  0.0
11     0001499588478181249                  0.0
12     0001521034365414265                  0.0
13     0001522706438705020                  0.0
14      000184052201924153                  0.0
15     0001875931252987660                  0.0
16     0001936363130317831                  0.0
17     0002032890182827978                  0.0
18     0002261098850688772                  0.0
19     0002585859302677487              

In [70]:
testOut(rfcPred)

             fullVisitorId  PredictedLogRevenue
0      0000059488412965267                  1.0
1      0000118334805178127                  1.0
2      0000174453501096099                  1.0
3      0000397214032106948                  1.0
4      0000421850492821864                  1.0
5      0000955020478109523                  1.0
6      0000998759681663618                  1.0
7      0001059349366430257                  1.0
8      0001251773762451924                  1.0
9      0001269799921883506                  1.0
10      000140003762682805                  1.0
11     0001499588478181249                  1.0
12     0001521034365414265                  1.0
13     0001522706438705020                  1.0
14      000184052201924153                  1.0
15     0001875931252987660                  1.0
16     0001936363130317831                  1.0
17     0002032890182827978                  1.0
18     0002261098850688772                  1.0
19     0002585859302677487              

C:\Users\aditya\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in expm1
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\aditya\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in expm1
  This is separate from the ipykernel package so we can avoid doing imports until


In [71]:
testOut(mlpPred)

C:\Users\aditya\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in expm1
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\aditya\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in expm1
  This is separate from the ipykernel package so we can avoid doing imports until


             fullVisitorId  PredictedLogRevenue
0      0000059488412965267                  1.0
1      0000118334805178127                  1.0
2      0000174453501096099                  1.0
3      0000397214032106948                  1.0
4      0000421850492821864                  1.0
5      0000955020478109523                  1.0
6      0000998759681663618                  1.0
7      0001059349366430257                  1.0
8      0001251773762451924                  1.0
9      0001269799921883506                  1.0
10      000140003762682805                  1.0
11     0001499588478181249                  1.0
12     0001521034365414265                  1.0
13     0001522706438705020                  1.0
14      000184052201924153                  1.0
15     0001875931252987660                  1.0
16     0001936363130317831                  1.0
17     0002032890182827978                  1.0
18     0002261098850688772                  1.0
19     0002585859302677487              

In [72]:
testOut(svrPred)

             fullVisitorId  PredictedLogRevenue
0      0000059488412965267                  0.0
1      0000118334805178127                  0.0
2      0000174453501096099                  0.0
3      0000397214032106948                  0.0
4      0000421850492821864                  0.0
5      0000955020478109523                  0.0
6      0000998759681663618                  0.0
7      0001059349366430257                  0.0
8      0001251773762451924                  0.0
9      0001269799921883506                  0.0
10      000140003762682805                  0.0
11     0001499588478181249                  0.0
12     0001521034365414265                  0.0
13     0001522706438705020                  0.0
14      000184052201924153                  0.0
15     0001875931252987660                  0.0
16     0001936363130317831                  0.0
17     0002032890182827978                  0.0
18     0002261098850688772                  0.0
19     0002585859302677487              